In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import numpy
import pandas
from IPython.display import display
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import custom

In [2]:
with open("data/korean_vector.pkl", mode = "rb") as f :
    vec_dict = pickle.load(f)

original_df = pandas.read_csv("data/korean_movie.txt", sep = "\t", encoding = "UTF8", index_col = 0)
val_df = pandas.read_csv("data/korean_movie_test.txt", sep = "\t", encoding = "UTF8", index_col = 0)

print(len(vec_dict))
display(original_df)
display(val_df)

30187


,document,label,word_len
0,아 더빙 진짜 짜증나네요 목소리,0,5
1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나,1,11
2,너 무재 밓었 다그 래서 보는것을 추천 한 다,0,9
3,교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정,0,9
4,사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만 했...,1,20
...,...,...,...
149995,인간 이 문제 지 소 는 뭔 죄인 가,0,9
149996,평점 이 너무 낮아서,1,4
149997,이 게 뭐 요 한국인 은 거들 먹거리 고 필리핀 혼혈 은 착하다,0,13
149998,청춘 영화 의 최고봉 방황 과 우울했던 날 들 의 자화상,1,11


,document,label,word_len
0,굳,1,1
1,NaN,0,0
2,뭐 야 이 평점 들 은 나쁘진 않지만 점 짜 리 는 더 더욱 아니잖아,0,15
3,지루하지는 않은데 완전 막장 임 돈 주고 보기 에는,0,9
4,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기 를 불편하게 하죠,0,14
...,...,...,...
49995,오랜 만 에 평점 로 긴 했네 킹왕짱 쌈뽕 한 영화 를 만났습니다 강렬하게 육 쾌함,1,16
49996,의지 박약 들 이나 하는거다 탈영 은 일단 주인공 김대희 닮았고 이등병 찐 따,0,14
49997,그림 도 좋고 완성 도도 높았지만 보는 내내 불안하게 만든다,0,10
49998,절대 봐 서는 안 될 영화 재미 도 없고 기분 만 잡 치고 한 세트 장 에서 다 해먹 네,0,20


In [3]:


df = original_df[(original_df['word_len'] > 0) & (original_df['word_len'] <= original_df['word_len'].mean() + original_df['word_len'].std())]
sentences = df['document']
t = df['label']
word_len = df['word_len'].max()

x = custom.word_vectorize(sentences, vec_dict, word_len)

print(len(x))
print(len(t))

129826
129826


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(x, dtype = torch.float, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)

zip_x = tensor_x
zip_t = tensor_t

dataloader = DataLoader(list(zip(zip_x, zip_t)),batch_size=200,shuffle=True)

In [11]:
##AI
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.rnn = nn.LSTM(200,10,batch_first = True,device = device)
        self.f = nn.Sequential(
            nn.Linear(10,2,device=device),
            nn.ReLU()
        )
    def forward(self, x) :
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x

F = NN()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 1)
epoch = 100



In [9]:
df2 = val_df[(val_df['word_len'] > 0) & (val_df['word_len'] <= val_df['word_len'].mean() + val_df['word_len'].std())]

sentences = df2["document"]
val_t = df2["label"]
word_len = df2["word_len"].max()

val_x = custom.word_vectorize(sentences, vec_dict, word_len)

val_x = torch.tensor(val_x, dtype = torch.float, device = device)
val_t = torch.tensor(val_t, dtype = torch.long, device = device)

zip_x = val_x
zip_t = val_t

val_loader = DataLoader(list(zip(zip_x, zip_t)),batch_size=1000,shuffle=False)